<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/japanese-roberta-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
MODEL = 'rinna/japanese-roberta-base'

In [2]:
!pwd

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
!ls drive/MyDrive/experiment_results

hs_txt_classification


In [6]:
SAVE_DIR = 'drive/MyDrive/experiment_results/hs_txt_classification'

In [7]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


In [8]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [9]:
#!pip install fugashi unidic_lite

In [10]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
import evaluate
print(evaluate.__version__)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

2.19.1
4.41.1
0.4.2


## 学習データ, テストデータ

In [11]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)

(4656, 5)


In [12]:
df_train = df[df['hold'] != 0]    # 学習データ
df_valid = df[df['hold'] == 0]    # 検証データ
df_train.shape, df_valid.shape

((3759, 5), (897, 5))

In [13]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [14]:
df_test = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_test.csv')
print(df_test.shape)

(600, 4)


In [15]:
ds_test = datasets.Dataset.from_pandas(df_test[['text','label']], preserve_index=False)

## トークナイザー

In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5TokenizerFast(name_or_path='rinna/japanese-roberta-base', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=128)    # 学習データの文長が短いので128に設定

In [18]:
ds_tkn_train = ds_train.map(tokenize_function, batched=True)
ds_tkn_valid = ds_valid.map(tokenize_function, batched=True)
ds_tkn_train, ds_tkn_valid

Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 3759
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 897
 }))

In [19]:
# textカラムの削除
ds_tkn_train = ds_tkn_train.remove_columns(['text'])
ds_tkn_valid = ds_tkn_valid.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_train = ds_tkn_train.rename_column('label', 'labels')
ds_tkn_valid = ds_tkn_valid.rename_column('label', 'labels')

In [20]:
ds_tkn_test = ds_test.map(tokenize_function, batched=True)
ds_tkn_test

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 600
})

In [21]:
# textカラムの削除
ds_tkn_test = ds_tkn_test.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_test = ds_tkn_test.rename_column('label', 'labels')

## 学習

In [22]:
metric = evaluate.load("f1")

# 精度評価関数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
def do_training():
  print('N_EPOCH: ', N_EPOCH)
  print('BATCH_SIZE: ', BATCH_SIZE)
  print('LR', LR)

  model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=2)

  trainer_args = TrainingArguments(
    seed=np.random.randint(0, 9999),
    output_dir=SAVE_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    #save_strategy="epoch",
    save_steps=1e6, # saveのステップを大きくしてここではモデルの保存を行わないようにする
    log_level="critical",
    num_train_epochs=N_EPOCH,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
    )

  trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=ds_tkn_train,
    eval_dataset=ds_tkn_valid,
    compute_metrics=compute_metrics,
    )

  #return trainer

  print('----------  Start Training  ----------')
  trainer.train()

  print('----------  Res on Valid  ----------')
  res_json = trainer.evaluate()
  print(res_json)

  print('----------  Res on Test  ----------')
  test_preds = trainer.predict(ds_tkn_test)
  test_metric = test_preds.metrics
  print(test_metric)

  return res_json, test_metric


In [24]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

Experiment Name:  japanese-roberta-base_Epoch3_Batch16_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at rinna/japanese-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------


Epoch,Training Loss,Validation Loss,F1
1,0.264100,0.215410,0.000000
2,0.216800,0.179682,0.000000
3,0.182600,0.157989,0.000000


----------  Res on Valid  ----------


{'eval_loss': 0.1579885482788086, 'eval_f1': 0.0, 'eval_runtime': 1.2831, 'eval_samples_per_second': 699.106, 'eval_steps_per_second': 58.454, 'epoch': 3.0}
----------  Res on Test  ----------
{'test_loss': 0.16525624692440033, 'test_f1': 0.0, 'test_runtime': 0.9671, 'test_samples_per_second': 620.408, 'test_steps_per_second': 51.701}


In [25]:
# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

drive/MyDrive/experiment_results/hs_txt_classification/VALID__japanese-roberta-base_Epoch3_Batch16_Lr3e-06.json
drive/MyDrive/experiment_results/hs_txt_classification/TEST__japanese-roberta-base_Epoch3_Batch16_Lr3e-06.json


In [26]:
!ls drive/MyDrive/experiment_results/hs_txt_classification

bert-large-japanese-v2	OLD   TEST__japanese-roberta-base_Epoch3_Batch16_Lr3e-06.json
checkpoint-940		OLD2  VALID__japanese-roberta-base_Epoch3_Batch16_Lr3e-06.json


In [27]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch16_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2256, 'grad_norm': 1.4946200847625732, 'learning_rate': 6.695035460992908e-06, 'epoch': 1.0}
{'eval_loss': 0.16313259303569794, 'eval_f1': 0.0392156862745098, 'eval_runtime': 1.6398, 'eval_samples_per_second': 547.015, 'eval_steps_per_second': 45.737, 'epoch': 1.0}
{'loss': 0.1511, 'grad_norm': 11.028458595275879, 'learning_rate': 3.3900709219858157e-06, 'epoch': 2.0}
{'eval_loss': 0.15963922441005707, 'eval_f1': 0.46511627906976744, 'eval_runtime': 1.6476, 'eval_samples_per_second': 544.444, 'eval_steps_per_second': 45.522, 'epoch': 2.0}
{'loss': 0.1183, 'grad_norm': 0.9388155341148376, 'learning_rate': 5.673758865248228e-08, 'epoch': 3.0}
{'eval_loss': 0.15834666788578033, 'eval_f1': 0.5087719298245614, 'eval_runtime': 1.2419, 'eval_samples_per_second': 722.287, 'eval_steps_per_second': 60.392, 'epoch': 3.0}
{'train_runtime': 82.0289, 'train_samples_per_second': 137.476, 'train_steps_per_second': 8.595, 'train_loss': 0.164993288838271

In [28]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch16_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2033, 'grad_norm': 0.35202983021736145, 'learning_rate': 2.0127659574468084e-05, 'epoch': 1.0}
{'eval_loss': 0.13658279180526733, 'eval_f1': 0.0, 'eval_runtime': 1.3948, 'eval_samples_per_second': 643.112, 'eval_steps_per_second': 53.772, 'epoch': 1.0}
{'loss': 0.1362, 'grad_norm': 2.689196825027466, 'learning_rate': 1.0170212765957446e-05, 'epoch': 2.0}
{'eval_loss': 0.18288442492485046, 'eval_f1': 0.5068493150684932, 'eval_runtime': 1.1828, 'eval_samples_per_second': 758.39, 'eval_steps_per_second': 63.411, 'epoch': 2.0}
{'loss': 0.0953, 'grad_norm': 0.22845345735549927, 'learning_rate': 1.7021276595744683e-07, 'epoch': 3.0}
{'eval_loss': 0.16901721060276031, 'eval_f1': 0.5252525252525252, 'eval_runtime': 1.1776, 'eval_samples_per_second': 761.7, 'eval_steps_per_second': 63.687, 'epoch': 3.0}
{'train_runtime': 92.4598, 'train_samples_per_second': 121.967, 'train_steps_per_second': 7.625, 'train_loss': 0.14492283002704592, 'epoch': 3.0

In [29]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch16_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.227, 'grad_norm': 0.3612994849681854, 'learning_rate': 3.3404255319148935e-05, 'epoch': 1.0}
{'eval_loss': 0.21634924411773682, 'eval_f1': 0.0, 'eval_runtime': 1.5798, 'eval_samples_per_second': 567.777, 'eval_steps_per_second': 47.473, 'epoch': 1.0}
{'loss': 0.2257, 'grad_norm': 0.35721802711486816, 'learning_rate': 1.673758865248227e-05, 'epoch': 2.0}
{'eval_loss': 0.21561016142368317, 'eval_f1': 0.0, 'eval_runtime': 1.6157, 'eval_samples_per_second': 555.167, 'eval_steps_per_second': 46.419, 'epoch': 2.0}
{'loss': 0.2239, 'grad_norm': 0.35210272669792175, 'learning_rate': 7.092198581560284e-08, 'epoch': 3.0}
{'eval_loss': 0.21164587140083313, 'eval_f1': 0.0, 'eval_runtime': 1.6899, 'eval_samples_per_second': 530.813, 'eval_steps_per_second': 44.382, 'epoch': 3.0}
{'train_runtime': 83.1274, 'train_samples_per_second': 135.659, 'train_steps_per_second': 8.481, 'train_loss': 0.2255082367159796, 'epoch': 3.0}
----------  Res on Valid  --

#### Batch Size -> 32

In [30]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch32_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.314, 'grad_norm': 0.7691614031791687, 'learning_rate': 2.0084745762711865e-06, 'epoch': 1.0}
{'eval_loss': 0.2147434949874878, 'eval_f1': 0.0, 'eval_runtime': 1.2213, 'eval_samples_per_second': 734.445, 'eval_steps_per_second': 61.408, 'epoch': 1.0}
{'loss': 0.224, 'grad_norm': 1.3698632717132568, 'learning_rate': 1.0084745762711866e-06, 'epoch': 2.0}
{'eval_loss': 0.20950081944465637, 'eval_f1': 0.0, 'eval_runtime': 1.2133, 'eval_samples_per_second': 739.315, 'eval_steps_per_second': 61.816, 'epoch': 2.0}
{'loss': 0.2161, 'grad_norm': 0.7969316840171814, 'learning_rate': 8.474576271186441e-09, 'epoch': 3.0}
{'eval_loss': 0.2007775604724884, 'eval_f1': 0.0, 'eval_runtime': 1.5944, 'eval_samples_per_second': 562.582, 'eval_steps_per_second': 47.039, 'epoch': 3.0}
{'train_runtime': 54.9446, 'train_samples_per_second': 205.243, 'train_steps_per_second': 6.443, 'train_loss': 0.2513439534074169, 'epoch': 3.0}
----------  Res on Valid  ------

In [31]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch32_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2395, 'grad_norm': 0.5672323703765869, 'learning_rate': 6.666666666666667e-06, 'epoch': 1.0}
{'eval_loss': 0.19284503161907196, 'eval_f1': 0.0, 'eval_runtime': 1.2569, 'eval_samples_per_second': 713.683, 'eval_steps_per_second': 59.673, 'epoch': 1.0}
{'loss': 0.1661, 'grad_norm': 7.600222110748291, 'learning_rate': 3.361581920903955e-06, 'epoch': 2.0}
{'eval_loss': 0.14978846907615662, 'eval_f1': 0.4745762711864407, 'eval_runtime': 1.641, 'eval_samples_per_second': 546.631, 'eval_steps_per_second': 45.705, 'epoch': 2.0}
{'loss': 0.1313, 'grad_norm': 2.143850803375244, 'learning_rate': 5.649717514124294e-08, 'epoch': 3.0}
{'eval_loss': 0.13762837648391724, 'eval_f1': 0.512396694214876, 'eval_runtime': 1.1931, 'eval_samples_per_second': 751.84, 'eval_steps_per_second': 62.863, 'epoch': 3.0}
{'train_runtime': 54.2658, 'train_samples_per_second': 207.81, 'train_steps_per_second': 6.523, 'train_loss': 0.17896419460490598, 'epoch': 3.0}
-----

In [32]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch32_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2292, 'grad_norm': 0.44932544231414795, 'learning_rate': 1.9999999999999998e-05, 'epoch': 1.0}
{'eval_loss': 0.20872774720191956, 'eval_f1': 0.0, 'eval_runtime': 1.2563, 'eval_samples_per_second': 714.026, 'eval_steps_per_second': 59.701, 'epoch': 1.0}
{'loss': 0.1542, 'grad_norm': 1.0130701065063477, 'learning_rate': 1.0084745762711864e-05, 'epoch': 2.0}
{'eval_loss': 0.13445435464382172, 'eval_f1': 0.5656565656565656, 'eval_runtime': 1.5916, 'eval_samples_per_second': 563.586, 'eval_steps_per_second': 47.123, 'epoch': 2.0}
{'loss': 0.1102, 'grad_norm': 3.5931649208068848, 'learning_rate': 8.47457627118644e-08, 'epoch': 3.0}
{'eval_loss': 0.14060154557228088, 'eval_f1': 0.537037037037037, 'eval_runtime': 1.2021, 'eval_samples_per_second': 746.198, 'eval_steps_per_second': 62.391, 'epoch': 3.0}
{'train_runtime': 56.4662, 'train_samples_per_second': 199.712, 'train_steps_per_second': 6.269, 'train_loss': 0.16450774198198048, 'epoch': 3.0

In [33]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch3_Batch32_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2312, 'grad_norm': 4.025142669677734, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
{'eval_loss': 0.1661057323217392, 'eval_f1': 0.4077669902912621, 'eval_runtime': 1.5954, 'eval_samples_per_second': 562.246, 'eval_steps_per_second': 47.011, 'epoch': 1.0}
{'loss': 0.1599, 'grad_norm': 1.5786988735198975, 'learning_rate': 1.6807909604519774e-05, 'epoch': 2.0}
{'eval_loss': 0.20373891294002533, 'eval_f1': 0.14814814814814814, 'eval_runtime': 1.2042, 'eval_samples_per_second': 744.894, 'eval_steps_per_second': 62.282, 'epoch': 2.0}
{'loss': 0.1438, 'grad_norm': 0.8988903760910034, 'learning_rate': 2.8248587570621473e-07, 'epoch': 3.0}
{'eval_loss': 0.14515987038612366, 'eval_f1': 0.5333333333333333, 'eval_runtime': 1.1711, 'eval_samples_per_second': 765.945, 'eval_steps_per_second': 64.042, 'epoch': 3.0}
{'train_runtime': 54.0255, 'train_samples_per_second': 208.735, 'train_steps_per_second': 6.552, 'train_loss': 0.178311094725872

#### Top3のエポック数UP

In [34]:
N_EPOCH = 5
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch5_Batch16_Lr3e-05
N_EPOCH:  5
BATCH_SIZE:  16
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2305, 'grad_norm': 0.45063552260398865, 'learning_rate': 2.4e-05, 'epoch': 1.0}
{'eval_loss': 0.21194924414157867, 'eval_f1': 0.0, 'eval_runtime': 1.8905, 'eval_samples_per_second': 474.467, 'eval_steps_per_second': 39.671, 'epoch': 1.0}
{'loss': 0.2123, 'grad_norm': 0.9340553879737854, 'learning_rate': 1.8051063829787234e-05, 'epoch': 2.0}
{'eval_loss': 0.2055957019329071, 'eval_f1': 0.0392156862745098, 'eval_runtime': 1.2499, 'eval_samples_per_second': 717.682, 'eval_steps_per_second': 60.007, 'epoch': 2.0}
{'loss': 0.2021, 'grad_norm': 4.231803894042969, 'learning_rate': 1.2051063829787234e-05, 'epoch': 3.0}
{'eval_loss': 0.19334253668785095, 'eval_f1': 0.26666666666666666, 'eval_runtime': 1.2193, 'eval_samples_per_second': 735.696, 'eval_steps_per_second': 61.513, 'epoch': 3.0}
{'loss': 0.1523, 'grad_norm': 0.8857033848762512, 'learning_rate': 6.051063829787234e-06, 'epoch': 4.0}
{'eval_loss': 0.16844335198402405, 'eval_f1': 0.47422

In [35]:
N_EPOCH = 5
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch5_Batch32_Lr3e-05
N_EPOCH:  5
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.218, 'grad_norm': inf, 'learning_rate': 2.4101694915254238e-05, 'epoch': 1.0}
{'eval_loss': 0.14442534744739532, 'eval_f1': 0.5426356589147288, 'eval_runtime': 1.2172, 'eval_samples_per_second': 736.913, 'eval_steps_per_second': 61.615, 'epoch': 1.0}
{'loss': 0.1386, 'grad_norm': 3.1582143306732178, 'learning_rate': 1.8101694915254238e-05, 'epoch': 2.0}
{'eval_loss': 0.18063867092132568, 'eval_f1': 0.4745762711864407, 'eval_runtime': 1.223, 'eval_samples_per_second': 733.419, 'eval_steps_per_second': 61.323, 'epoch': 2.0}
{'loss': 0.0921, 'grad_norm': 6.81752347946167, 'learning_rate': 1.2101694915254238e-05, 'epoch': 3.0}
{'eval_loss': 0.23018893599510193, 'eval_f1': 0.4938271604938272, 'eval_runtime': 1.6391, 'eval_samples_per_second': 547.252, 'eval_steps_per_second': 45.757, 'epoch': 3.0}
{'loss': 0.0543, 'grad_norm': 0.08874143660068512, 'learning_rate': 6.101694915254238e-06, 'epoch': 4.0}
{'eval_loss': 0.22045139968395233, 'eval_

In [36]:
N_EPOCH = 5
BATCH_SIZE = 32
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  japanese-roberta-base_Epoch5_Batch32_Lr5e-05
N_EPOCH:  5
BATCH_SIZE:  32
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.248, 'grad_norm': 0.7087531685829163, 'learning_rate': 4.0423728813559324e-05, 'epoch': 1.0}
{'eval_loss': 0.19275133311748505, 'eval_f1': 0.13793103448275865, 'eval_runtime': 1.6648, 'eval_samples_per_second': 538.792, 'eval_steps_per_second': 45.049, 'epoch': 1.0}
{'loss': 0.1808, 'grad_norm': 1.2007495164871216, 'learning_rate': 3.042372881355932e-05, 'epoch': 2.0}
{'eval_loss': 0.13056538999080658, 'eval_f1': 0.36111111111111116, 'eval_runtime': 1.1852, 'eval_samples_per_second': 756.814, 'eval_steps_per_second': 63.279, 'epoch': 2.0}
{'loss': 0.1463, 'grad_norm': 1.1545600891113281, 'learning_rate': 2.0423728813559322e-05, 'epoch': 3.0}
{'eval_loss': 0.15448793768882751, 'eval_f1': 0.5148514851485149, 'eval_runtime': 1.2162, 'eval_samples_per_second': 737.533, 'eval_steps_per_second': 61.667, 'epoch': 3.0}
{'loss': 0.1185, 'grad_norm': 0.4708322286605835, 'learning_rate': 1.0423728813559321e-05, 'epoch': 4.0}
{'eval_loss': 0.137053